In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
import numpy as np

from summer2 import CompartmentalModel
from summer2.parameters import Parameter, DerivedOutput

In [ ]:
config = {
    "tot_popn": 156400.,
    "C_0": 6400.,
    "end_time": 21. * 52.,
    "t_step": 3.,
}

parameters = {
    "k": 1. / 150000.,
    "ave_infous": 3.,
    "birth": 1100.,
}

In [ ]:
def build_si_model(
    model_config: dict,
) -> CompartmentalModel:

    # Model characteristics
    compartments = ("Susceptible", "Infectious")
    model = CompartmentalModel(
        times=(0, model_config["end_time"]),
        compartments=compartments,
        infectious_compartments=["Infectious"],
        timestep=model_config["t_step"],
    )
    model.set_initial_population(
        distribution={
            "Susceptible": model_config["tot_popn"] - model_config["C_0"],
            "Infectious": model_config["C_0"],
        }
    )
    
    # Transitions
    model.add_infection_density_flow(
        name="infection", 
        contact_rate=Parameter("k"),
        source="Susceptible", 
        dest="Infectious",
    )
    model.add_death_flow(
        name="infection_death",
        death_rate=1. / Parameter("ave_infous"),
        source="Infectious",
    )
    model.add_importation_flow(
        name="birth",
        num_imported=Parameter("birth"),
        dest="Susceptible",
        split_imports=True,
    )

    # Outputs
    model.request_output_for_compartments(
        name="total_population",
        compartments=compartments,
        save_results=False,
    )
    model.request_output_for_compartments(
        name="susceptible",
        compartments=("Susceptible",),
        save_results=False,
    )
    model.request_function_output(
        name="susc_prop",
        func=DerivedOutput("susceptible") / DerivedOutput("total_population"),
    )
    return model

In [ ]:
si_model = build_si_model(config)
si_model.run(parameters=parameters)
compartment_values = si_model.get_outputs_df()
derived_outputs = si_model.get_derived_outputs_df()

#### Unnumbered figure on the left

In [ ]:
fig, (left_ax, right_ax) = plt.subplots(1, 2, figsize=(14, 5))

left_ax.plot(si_model.times, compartment_values["Infectious"], color="r", label="Susceptible")
left_ax.set_xlabel("Time (days)")
left_ax.set_xlim(0, 20 * 365)
# left_ax.set_xticks(range(0, 21900, 3650))
left_ax.xaxis.set_minor_locator(AutoMinorLocator(2))
# left_ax.set_ylim(0., 1e5)
# left_ax.set_ylabel("Number of susceptible or immune persons")
# left_ax.yaxis.set_minor_locator(AutoMinorLocator(5))
# left_ax.spines.top.set_visible(False)
# left_ax.legend()

# twin_ax = left_ax.twinx()
# twin_ax.set_ylim(0, 300.)
# twin_ax.set_ylabel("Number of new infectious persons per time step")
# twin_ax.yaxis.set_minor_locator(AutoMinorLocator(2))
# twin_ax.spines.top.set_visible(False)
# twin_ax.legend(loc=6)

# image = plt.imread("./figures/model_4_3a_fig.JPG")
# right_ax.axis("off")
# right_ax.imshow(image, aspect="auto");